In [1]:
import numpy as np
import pandas as pd

df_features = pd.read_parquet("data/features_d002.parquet")
df_label = pd.read_parquet("data/label_d002.parquet")
print(df_label.shape)
df_label.head()

(715680, 5)


,close,ret,trgt,bin,side
index,,,,,
2021-01-01 00:00:00+00:00,29016.23,NaN,NaN,0.0,0.0
2021-01-01 00:03:00+00:00,28965.02,NaN,NaN,0.0,0.0
2021-01-01 00:06:00+00:00,28928.06,NaN,NaN,0.0,0.0
2021-01-01 00:09:00+00:00,28851.82,NaN,NaN,0.0,0.0
2021-01-01 00:12:00+00:00,28786.75,NaN,NaN,0.0,0.0


In [2]:
meta_mask = df_label["ret"].notna().tolist()
side_mask = (df_label["side"].astype(int) != 0).tolist()

meta_label = df_label[meta_mask]["bin"].astype(np.int8).reset_index(drop=True)
meta_label.name = "label"
side_label = (
    (df_label[side_mask]["side"].astype(int) == 1)
    .astype(np.int8)
    .reset_index(drop=True)
)
side_label.name = "label"

meta_features = df_features[meta_mask].reset_index(drop=True)
side_features = df_features[side_mask].reset_index(drop=True)

assert meta_features.shape[0] == meta_label.shape[0]
assert side_features.shape[0] == side_label.shape[0]

# invalid_len = 300
# meta_features = meta_features.iloc[invalid_len:]
# meta_label = meta_label.iloc[invalid_len:]
# side_features = side_features.iloc[invalid_len:]
# side_label = side_label.iloc[invalid_len:]

# meta_split = int(meta_features.shape[0] * 0.8)
# side_split = int(side_features.shape[0] * 0.8)

# meta_x_train = meta_features.iloc[:meta_split]
# meta_x_test = meta_features.iloc[meta_split:]
# meta_y_train = meta_label.iloc[:meta_split]
# meta_y_test = meta_label.iloc[meta_split:]

# assert meta_x_train.shape[0] == meta_y_train.shape[0]
# assert meta_x_test.shape[0] == meta_y_test.shape[0]

# side_x_train = side_features.iloc[:side_split]
# side_x_test = side_features.iloc[side_split:]
# side_y_train = side_label.iloc[:side_split]
# side_y_test = side_label.iloc[side_split:]

# assert side_x_train.shape[0] == side_y_train.shape[0]
# assert side_x_test.shape[0] == side_y_test.shape[0]

In [3]:
meta_label.value_counts()

label
0    499394
1     34693
Name: count, dtype: int64

In [5]:
# meta feature selection
from os import cpu_count
from BorutaShap import BorutaShap
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(
    n_jobs=cpu_count() - 1, class_weight={0: 1, 1: 14}, max_depth=10, n_estimators=500
)
meta_feat_selector = BorutaShap(rf, importance_measure="shap", classification=True)
meta_feat_selector.fit(
    X=meta_features,
    y=meta_label,
    n_trials=100,
    sample=False,
    train_or_test="test",
    normalize=True,
    verbose=True,
)

  0%|          | 0/100 [00:00<?, ?it/s]

: 